In [1]:
%env CUBLAS_WORKSPACE_CONFIG=:16:8

env: CUBLAS_WORKSPACE_CONFIG=:16:8


In [2]:
from autoencodix.utils.example_data import (
    EXAMPLE_MULTI_BULK,
    EXAMPLE_MULTI_SC,
    EXAMPLE_PROCESSED_DATA,
)

# EXAMPLE_DATA hold PyTorch Datasets (child with extra info) with metdata for train, test and valid splits
processed_data = EXAMPLE_PROCESSED_DATA
raw_bulk = EXAMPLE_MULTI_BULK
raw_sc = EXAMPLE_MULTI_SC

/Users/maximilianjoas/development/autoencodix_package/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
EXAMPLE_PROCESSED_DATA.train.metadata

,cluster,age,size,density,category,random_feature,sample_id
0,0,21.194224,4.538300,0.113699,A,-0.657083,sample_303
1,0,23.930016,6.864434,0.086318,A,1.280058,sample_259
2,3,53.074559,12.812042,1.111816,D,-0.046288,sample_717
3,4,53.380117,12.875572,0.525779,C,0.578705,sample_127
4,0,33.724569,2.429471,0.038608,B,0.657419,sample_576
...,...,...,...,...,...,...,...
694,0,28.165998,6.223125,0.286763,E,0.047880,sample_578
695,1,31.830988,4.929979,0.456122,B,0.126054,sample_946
696,2,40.607502,6.923138,2.313312,B,-0.288755,sample_284
697,0,40.449944,0.831899,0.096780,C,1.070726,sample_462


In [4]:
import autoencodix as acx
from autoencodix.utils.default_config import DefaultConfig

In [ ]:
vanillix = acx.Vanillix(config=DefaultConfig(epochs=200, learning_rate=0.005,n_layers=1, latent_dim=2, checkpoint_interval=50), data=processed_data)
vanillix.run()
vanillix.show_result()

In [ ]:
categorical_clusters = ["Cluster_"+str(c) for c in EXAMPLE_PROCESSED_DATA.train.metadata.cluster]
categorical_clusters

In [ ]:
vanillix._visualizer.show_latent_space(result=vanillix.result,plot_type="2D-scatter", split="train", label_list=categorical_clusters, param=["Cluster"])
## Problem: during training sample order is shuffled due to batching. Order cannot be recovered with current result object.

In [ ]:
vanillix.result.model

## Ontix

In [5]:
import autoencodix as acx
from autoencodix.utils.default_config import DefaultConfig

from autoencodix.utils.example_data import (
    EXAMPLE_MULTI_BULK,
    EXAMPLE_MULTI_SC,
    EXAMPLE_PROCESSED_DATA,
)

# EXAMPLE_DATA hold PyTorch Datasets (child with extra info) with metdata for train, test and valid splits
processed_data = EXAMPLE_PROCESSED_DATA


In [6]:

ont_lvl1 = dict()
ont_lvl2 = dict()

ont_lvl1["pwy-1"] = ["sub-pwy-1", "sub-pwy-2"]
ont_lvl1["pwy-2"] = ["sub-pwy-2"]
ont_lvl1["pwy-3"] = ["sub-pwy-1", "sub-pwy-3"]
# first third of feature ids in processed_data.train.feature_ids
ont_lvl2["sub-pwy-1"] = processed_data.train.feature_ids[: int(len(processed_data.train.feature_ids) / 3)]
# second third of feature ids in processed_data.train.feature_ids
ont_lvl2["sub-pwy-2"] = processed_data.train.feature_ids[
	int(len(processed_data.train.feature_ids) / 3) : int(2 * len(processed_data.train.feature_ids) / 3)
]
# last third of feature ids in processed_data.train.feature_ids
ont_lvl2["sub-pwy-3"] = processed_data.train.feature_ids[
	int(2 * len(processed_data.train.feature_ids) / 3) : int(len(processed_data.train.feature_ids))
]

# ont_lvl2["sub-pwy-1"] = ["gene-1", "gene-2"]
# ont_lvl2["sub-pwy-2"] = ["gene-3", "gene-4"]
# ont_lvl2["sub-pwy-3"] = ["gene-2", "gene-6"]

ontology_tuple = ( ont_lvl1, ont_lvl2 )


In [7]:
# Write each dictionary in ontology_tuple to a separate text file
for i, ont_dict in enumerate(ontology_tuple):
	file_name = f"ontology_level_{i+1}.txt"
	with open(file_name, "w") as f:
		for key, values in ont_dict.items():
			for value in values:
				f.write(f"{value}\t{key}\n")
print("Ontology dictionaries written to ontology_level_1.txt and ontology_level_2.txt")

Ontology dictionaries written to ontology_level_1.txt and ontology_level_2.txt


In [8]:
ont_files = ["ontology_level_1.txt", "ontology_level_2.txt"]

In [9]:
ontix = acx.Ontix(ontologies=ont_files, sep="\t", config=DefaultConfig(epochs=5, learning_rate=0.005,n_layers=1), data=processed_data)
# ontix = acx.Ontix(ontologies=ontology_tuple, config=DefaultConfig(epochs=500, learning_rate=0.005,n_layers=1), user_data=processed_data)

In [10]:
ontix.run()

preprocessing
Ontix checks:
All possible feature names length: 30
Feature order length: 30
Feature names without filtering: 30
Mask layer 0 with shape torch.Size([3, 3]) and 5.0 connections
Mask layer 1 with shape torch.Size([30, 3]) and 30.0 connections
Latent Dim: 3
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
applying ontology masking
Epoch 1 - Train Loss: 857579.0312
Sub-losses: recon_loss: 857578.9453, var_loss: 1266.8643, anneal_factor: 0.0000, effective_beta_factor: 0.0000
Epoch 1 - Vali

Result Object Public Attributes:
------------------------------
latentspaces: TrainingDynamics object
sample_ids: TrainingDynamics object
reconstructions: TrainingDynamics object
mus: TrainingDynamics object
sigmas: TrainingDynamics object
losses: TrainingDynamics object
sub_losses: LossRegistry(_losses={'recon_loss': TrainingDynamics(), 'var_loss': TrainingDynamics(), 'anneal_factor': TrainingDynamics(), 'effective_beta_factor': TrainingDynamics()})
preprocessed_data: Tensor of shape (0,)
model: OntixArchitecture
model_checkpoints: TrainingDynamics object
datasets: DatasetContainer(train=<autoencodix.data._numeric_dataset.NumericDataset object at 0x33c4c6d70>, valid=<autoencodix.data._numeric_dataset.NumericDataset object at 0x33c4c6da0>, test=<autoencodix.data._numeric_dataset.NumericDataset object at 0x33c6749a0>)
new_datasets: DatasetContainer(train=None, valid=None, test=None)
adata_latent: AnnData object with n_obs × n_vars = 201 × 3
    uns: 'var_names'
final_reconstruction: <au

In [ ]:
ontix._visualizer.show_weights()

In [ ]:
ontix.show_result()

In [ ]:
categorical_clusters = ["Cluster_"+str(c) for c in EXAMPLE_PROCESSED_DATA.train.metadata.cluster]
categorical_clusters
ontix._visualizer.show_latent_space(result=ontix.result,plot_type="2D-scatter", split="train", label_list=categorical_clusters, param="Cluster")

In [ ]:
ontix.ontologies

In [ ]:
enumerate(ontix.result.model.ontologies)

In [ ]:
ontix.result.model.ontologies[1].keys()

In [ ]:
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt

masks = ontix.result.model.masks

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
for i, mask in enumerate(masks):
	sns.heatmap(np.array(mask), ax=axes[i], cmap="viridis")
	axes[i].set_title(f"Mask {i+1}")
plt.tight_layout()
plt.show()

In [ ]:
ontix.preprocessed_data.train.feature_ids

In [ ]:
# Print all available attributes of ontix
print("Attributes of ontix:")
for attr in dir(ontix):
	if not attr.startswith("__"):
		print(attr)


In [ ]:
import seaborn as sns
import pandas as pd
import umap
import matplotlib.pyplot as plt


# Fit UMAP to reduce to 2D
umap_2d_seaborn = umap.UMAP(n_components=2, random_state=42)
embedding_seaborn = umap_2d_seaborn.fit_transform(EXAMPLE_PROCESSED_DATA.train.data)

# Create a DataFrame for plotting
umap_df = pd.DataFrame({
	"UMAP1": embedding_seaborn[:, 0],
	"UMAP2": embedding_seaborn[:, 1],
	"Cluster": categorical_clusters
})

plt.figure(figsize=(8, 6))
sns.scatterplot(
	data=umap_df,
	x="UMAP1",
	y="UMAP2",
	hue="Cluster",
	palette="tab10",
	s=20,
	alpha=0.7
)
plt.title("2D UMAP projection of train data (colored by cluster_labels)")
plt.legend(title="Cluster", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()